<a href="https://colab.research.google.com/github/Tadashi1999/Repo_tst/blob/master/%E3%83%AD%E3%83%BC%E3%82%BD%E3%82%AF%E8%B6%B3%2BMACD%E4%BB%96.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#ローソク足チャート（簡略）
!pip install pandas-datareader

#pandas-datareaderによる株価の取得
import pandas as pd
import pandas_datareader as pdr
pdr=pdr.stooq.StooqDailyReader(symbols='4563.jp',start='Oct-01-2019',end="OCT-26-2020").read().sort_values(by='Date',ascending=True)
df_t=pdr


In [ ]:
df_t.tail(20)

In [ ]:
#Talibのインストール
! wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzvf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
!pip install Ta-lib
import talib as ta

In [ ]:
#ローソク足を簡易表示（実行可能）
import numpy as np
import plotly.graph_objects as go

def plot_chart(df):
  x=np.arange(df.shape[0])
  interval=1
  vals=np.arange(df.shape[0],step=interval)
  labels=list(df[::interval].index)

  fig=go.Figure(
      data=[go.Candlestick(
            x=x,
            open=np.array(df['Open']),
            high=np.array(df['High']),
            low=np.array(df['Low']),
            close=np.array(df['Close']),
            hovertext=['Date']),
           ],
      layout=go.Layout(
            xaxis=dict(
                tickvals=vals,
                ticktext=labels,
                tickangle=-45
            ),
      )
  )
  fig.show()
plot_chart(df_t)  

In [ ]:
#複数データ表示
import talib as ta
import numpy as np
import pandas as pd
!pip install plotly --upgrade

import plotly.offline as pyo
#import chart_studio.plotly as py
import plotly.graph_objects as go

#株価取得
import pandas_datareader.data as web
from datetime import datetime
start=datetime(2019,10,1)
end=datetime(2020,10,23)
df=web.DataReader('4563.jp','stooq',start,end)
#df=web.DataReader('^N225','yahoo','2019-10-01')
#df.tail()


hnp=np.array(df['High'])#npアレイ形式に変換
lnp=np.array(df['Low'])
cnp=np.array(df['Close'])
onp=np.array(df['Open'])
c=df['Close']


#単純移動平均
sma5=ta.SMA(c,timeperiod=2)
sma20=ta.SMA(c,timeperiod=10)
sma50=ta.SMA(c,timeperiod=50)

#Plotlyでは DataFrameを使うので、以下の追加
df['SMA5']=sma5
df['SMA20']=sma20
df['SMA50']=sma50

#MACD
macd,macd_sig,macd_hist=ta.MACD(c,fastperiod=5,slowperiod=10,signalperiod=9)
df["MACD"]=macd
df["MACD_SIG"]=macd_sig
df["MACD_HIST"]=macd_hist

#Plotlyでチャート
layout={
    'hight':1000,
    'title':{'text':'アンジェス','x':0.5},
    'titlefont':{'size':25},
    'xaxis':{'title':"",'rangeslider':{'visible':False}},
    'yaxis':{'domain':[.55,1],'title':"株価（円)",'side':"left",'tickformat':','},
    'yaxis2':{'domain':[.45,.55],'title':"MACD",'side':"right"},
}
#ローソク足
trace1=go.Candlestick(
    x=df.index,open=onp,high=hnp,low=lnp,close=cnp,yaxis="y2",name="Anges"
)
trace2=go.Scatter(yaxis="y2",x=df.index,y=df["SMA5"],name='SMA5',line=dict(color='Black',width=1.5))
trace3=go.Scatter(yaxis="y2",x=df.index,y=df["SMA20"],name='SMA20',line=dict(color='Black',width=0.5))
trace4=go.Scatter(yaxis="y2",x=df.index,y=df["SMA50"],name='SMA50',line=dict(color='darkblue',width=1))
trace5=go.Scatter(yaxis="y2",x=df.index,y=df["MACD"],name='MACD',line=dict(color='blue',width=1.5))
trace6=go.Scatter(yaxis="y2",x=df.index,y=df["MACD_SIG"],name='MACD_SIG',line=dict(color='red',width=1))
data=[trace1,trace2,trace3,trace4,trace5,trace6]
go.Figure(data).show()

Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.12.0)
